In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd

plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
def load(filename):
    x = pd.read_csv(os.path.join('../data/jhu/csse_covid_19_data/', filename))
    x = x.drop(columns=['Lat', 'Long']).set_index(['Country/Region', 'Province/State']).T
    x.index = pd.to_datetime(x.index, dayfirst=False)
    x.index.name = 'Date'
    return x.groupby(axis='columns', level='Country/Region').sum()
confirmed = load('csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered = load('csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
deaths = load('csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [ ]:
lower_bound = (deaths.where(deaths > 100) / confirmed).dropna(axis='columns', how='all')
lower_bound.plot(legend='best')


In [ ]:
upper_bound = (deaths.where(deaths > 100) / recovered).dropna(axis='columns', how='all')
upper_bound = np.minimum(upper_bound, 1.0)
upper_bound.plot(legend='best')


In [ ]:
pd.DataFrame({'lower_bound': lower_bound.iloc[-1],
              'upper_bound': upper_bound.iloc[-1]}).sort_values(by='lower_bound', ascending=False)

In [ ]:
for idx, col in lower_bound.iteritems():
    plt.figure(str(idx))
    plt.plot(pd.DataFrame({'lower':col, 'upper':upper_bound.loc[:,idx]}))
    plt.title(idx)
    plt.ylim(0, 0.15)

In [ ]:
all_deaths = deaths.sum(axis='columns')
all_deaths = all_deaths.where(all_deaths > 100)
all_confirmed = confirmed.sum(axis='columns')
all_recovered = recovered.sum(axis='columns')
all_recovered = all_recovered.where(all_recovered > 100)
pd.DataFrame({'lower_bound': all_deaths / all_confirmed,
              'upper_bounds': all_deaths / all_recovered}).plot()

